# Tensorflow (Keras)

In [1]:
# import packages that we'd work with
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a pandas dataframe.
df = pd.read_csv(r"c:\Users\USER\Desktop\MyDatasets\heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [3]:
# Replace the categorical attributes with numbers.
df.Sex.replace(['F', 'M'], [0, 1], inplace=True)
df.ChestPainType.replace(['ATA', 'NAP', 'ASY', 'TA'], [0, 1, 2, 3], inplace=True)
df.RestingECG.replace(['Normal', 'ST', 'LVH'], [0, 1, 2], inplace=True)
df.ExerciseAngina.replace(['N', 'Y'], [0, 1], inplace=True)
df.ST_Slope.replace(['Up', 'Flat', 'Down'], [0, 1, 2], inplace=True)
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,0,140,289,0,0,172,0,0.0,0,0
1,49,0,1,160,180,0,0,156,0,1.0,1,1
2,37,1,0,130,283,0,1,98,0,0.0,0,0
3,48,0,2,138,214,0,0,108,1,1.5,1,1
4,54,1,1,150,195,0,0,122,0,0.0,0,0


In [5]:
# Convert the dataframe into a numpy array, so we can train on it.
data = df.to_numpy()

# Slice the data into features and targets.
X = data[:, :11]
Y = data[:, 11]

# Split the data into training and testing chunks.
X_train, X_test, Y_train, Y_test = tts(X, Y, test_size=0.2, random_state=2)

# Scale/normalize the data
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [6]:
# Define the input_shape.
input_shape = (X_train_scaled.shape[1],)

# Build the model.
model = Sequential()
model.add(Dense(41, activation='relu', input_shape=input_shape))
#model.add(Dropout(0.2))
model.add(Dense(112, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 41)                492       
_________________________________________________________________
dense_1 (Dense)              (None, 112)               4704      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 113       
Total params: 5,309
Trainable params: 5,309
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Compile the model and define the loss and optimizer functions.
model.compile(loss='binary_crossentropy', 
                         optimizer=SGD(learning_rate=0.1), 
                         metrics=['accuracy'])

In [11]:
# Call model.fit on the training chunk, define batch size, epochs and  the validation data.
model.fit(X_train_scaled, Y_train,
                               batch_size=40, 
                               epochs=5,
                               verbose=2,
                               validation_data=(X_test_scaled, Y_test))
# Training!!!

Epoch 1/5
19/19 - 0s - loss: 0.1267 - accuracy: 0.9496 - val_loss: 0.4952 - val_accuracy: 0.8533
Epoch 2/5
19/19 - 0s - loss: 0.1105 - accuracy: 0.9591 - val_loss: 0.4888 - val_accuracy: 0.8587
Epoch 3/5
19/19 - 0s - loss: 0.1041 - accuracy: 0.9646 - val_loss: 0.4960 - val_accuracy: 0.8370
Epoch 4/5
19/19 - 0s - loss: 0.1001 - accuracy: 0.9687 - val_loss: 0.5025 - val_accuracy: 0.8424
Epoch 5/5
19/19 - 0s - loss: 0.0975 - accuracy: 0.9714 - val_loss: 0.5039 - val_accuracy: 0.8533


I've made up two samples of data, the first is a heart disease positive patient, the second is a heart disease negative patient<br>
Let's see if our model can tell.

In [12]:
# I standardised the data prior to.
patient_data = np.array([[-0.5789,  0.5220,  0.6349, -0.9879, -1.7750,  1.8279, -0.7600, -0.3676,
         -0.8088, -0.8119,  0.6583], [-1.3316,  0.5216,  0.6352, -1.1563,  0.4743, -0.5473, -0.7599,  0.1890,
         -0.8082, -0.8111, -1.0494]])

In [13]:
import tensorflow as tf
predicted = model.predict(tf.convert_to_tensor(patient_data))
for i in range(len(predicted)):
    prediction = (predicted[i]).round()
    if prediction==1:
        print(f'Patient {i+1} has a heart disease')
    else:
        print(f'Patient {i+1} does not have a heart disease')

Patient 1 has a heart disease
Patient 2 does not have a heart disease


Ok, with 86% accuracy, the model still predicted the sample perfectly.

In [ ]:
# ifunanyaScript